In [56]:
import pandas as pd
from pyspark.sql import functions as F, types as T

file = 'Dataset_3_sales.csv'

def get_spark_env(version, job_name, n):
    if version in ["2.4"]:
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.master("local[*]").config('spark.executor.memoryOverhead.config', n).appName(job_name).getOrCreate()
        return spark
    elif version in ["1.6"]:
        return None, None

def printDf(DF):
    return pd.DataFrame(DF.show(), columns=DF.columns)

def printRows(DF,NbRows):
    return pd.DataFrame(DF.take(NbRows), columns=DF.columns)

job_name = 'Formation'
spark = get_spark_env("2.4", job_name, '4g')
print (spark.version)

3.1.1


In [71]:
df = spark.read.options(inferSchema='True',delimiter='|').csv(file, header=True)
delColumn(df)
display(df)
df.printSchema()
df.count()

DataFrame[_c0: int, Unnamed: 0: int, Unnamed: 0.1: int, Unnamed: 0.1.1: int, Unnamed: 0.1.1.1: int, Unnamed: 0.1.1.1.1: string, Unnamed: 0.1.1.1.1.1: string, Unnamed: 0.1.1.1.1.1.1: string, Unnamed: 0.1.1.1.1.1.1.1: string, index: string, Item_Identifier: string, Item_Weight: string, Item_Fat_Content: string, Item_Visibility: double, Item_Type: string, Item_MRP: double, Outlet_Identifier: string, Outlet_Establishment_Year: int, Outlet_Size: string, Outlet_Location_Type: string, Outlet_Type: string, Item_Outlet_Sales: double]

root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- Unnamed: 0.1: integer (nullable = true)
 |-- Unnamed: 0.1.1: integer (nullable = true)
 |-- Unnamed: 0.1.1.1: integer (nullable = true)
 |-- Unnamed: 0.1.1.1.1: string (nullable = true)
 |-- Unnamed: 0.1.1.1.1.1: string (nullable = true)
 |-- Unnamed: 0.1.1.1.1.1.1: string (nullable = true)
 |-- Unnamed: 0.1.1.1.1.1.1.1: string (nullable = true)
 |-- index: string (nullable = true)
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: string (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outle

14204

In [69]:
def delColumn(dataFrame):
    dataFrame = dataFrame.drop('Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 
         'Unnamed: 0.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1.1', 'Unnamed: 0.1.1.1.1.1.1.1.1')
    return dataFrame

delColumn(df)

DataFrame[_c0: int, index: string, Item_Identifier: string, Item_Weight: string, Item_Fat_Content: string, Item_Visibility: double, Item_Type: string, Item_MRP: double, Outlet_Identifier: string, Outlet_Establishment_Year: int, Outlet_Size: string, Outlet_Location_Type: string, Outlet_Type: string, Item_Outlet_Sales: double]

In [73]:
def getEstablishmentByYear(dataFrame, column):
    delColumn(dataFrame)
    dataFrame = dataFrame.filter(df[column] > 2000)
    return printRows(dataFrame, 10)
getEstablishmentByYear(df, 'Outlet_Establishment_Year')

,_c0,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,index,...,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,1,1,1,1,1,1,1,1,1.0,1,...,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
1,5,5,5,5,5,5,5,5,5.0,5,...,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
2,8,8,8,8,8,8,8,8,8.0,8,...,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,None,Tier 2,Supermarket Type1,1076.5986
3,9,9,9,9,9,9,9,9,9.0,9,...,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,None,Tier 2,Supermarket Type1,4710.5350
4,16,16,16,16,16,16,16,16,16.0,16,...,Low Fat,0.008596,Health and Hygiene,115.3492,OUT018,2009,Medium,Tier 3,Supermarket Type2,1621.8888
5,19,19,19,19,19,19,19,19,19.0,19,...,Low Fat,0.102492,Dairy,230.5352,OUT035,2004,Small,Tier 2,Supermarket Type1,2748.4224
6,22,22,22,22,22,Allez au boulot ! :),22,22,22.0,22,...,Low Fat,0.025698,Household,196.5084,OUT035,2004,Small,Tier 2,Supermarket Type1,1587.2672
7,25,25,25,25,25,25,25,25,25.0,25,...,Low Fat,0.099887,Household,45.9060,OUT017,2007,None,Tier 2,Supermarket Type1,838.9080
8,26,26,26,26,26,26,26,26,26.0,26,...,Regular,0.066693,Snack Foods,42.3112,OUT035,2004,Small,Tier 2,Supermarket Type1,1065.2800
9,31,31,31,31,31,31,31,31,31.0,31,...,Low Fat,0.080829,Health and Hygiene,96.4436,OUT018,2009,Medium,Tier 3,Supermarket Type2,2741.7644


In [60]:
def getItemByType(dataFrame, column, itemType):
    dataFrame = dataFrame.filter(df[column] == itemType)
    return printRows(dataFrame, 10)
getItemByType(df, 'Item_Type', 'Canned')

,_c0,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,index,...,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,29,29,29,29,29,29,29,29,29.0,29,...,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
1,43,43,43,43,43,43,43,43,43.0,43,...,Low Fat,0.069103,Canned,259.9278,OUT018,2009,Medium,Tier 3,Supermarket Type2,6768.5228
2,44,44,44,44,44,44,44,44,44.0,44,...,Regular,0.042278,Canned,126.5046,OUT013,1987,High,Tier 3,Supermarket Type1,373.5138
3,47,47,47,47,47,47,47,47,47.0,47,...,Low Fat,0.025880,Canned,265.2226,OUT045,2002,None,Tier 2,Supermarket Type1,5815.0972
4,52,52,52,52,52,52,52,52,52.0,52,...,Regular,0.028697,Canned,117.0492,OUT013,1987,High,Tier 3,Supermarket Type1,810.9444
5,59,59,59,59,59,59,59,59,59.0,59,...,Low Fat,0.061082,Canned,180.0344,OUT019,1985,Small,Tier 1,Grocery Store,892.1720
6,73,73,73,73,73,73,73,73,73.0,73,...,Low Fat,0.011325,Canned,189.6188,OUT017,2007,None,Tier 2,Supermarket Type1,2285.0256
7,79,79,79,79,79,79,79,79,79.0,79,...,Low Fat,0.000000,Canned,222.5456,OUT045,2002,None,Tier 2,Supermarket Type1,5305.0944
8,105,105,105,105,105,105,105,105,105.0,105,...,Regular,0.046770,Canned,141.1838,OUT013,1987,High,Tier 3,Supermarket Type1,2247.7408
9,122,122,122,122,122,122,122,122,122.0,122,...,Regular,0.171939,Canned,92.5120,OUT010,1998,None,Tier 3,Grocery Store,186.4240


In [79]:
from pyspark.sql.functions import mean as _mean

def meanSales(dataFrame, column):
    dataFrame.select(_mean(dataFrame(column)))
meanSales(df, 'Item_Outlet_Sales')

TypeError: 'DataFrame' object is not callable